# 세이프존과 캐릭터 거리 계산용 데이터 전처리

#### 데이터 불러오기 및 저장

In [1]:
#library
import pandas as pd
from datetime import datetime
import swifter


c:\Users\PC\Desktop\AI_Lab\pubg\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#파일 불러오기
df = pd.read_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\data\tel0.csv')

#필요한 컬럼별로 dataframe 구성
df1=df[['_D','character.name', 'character.location.x', 'character.location.y', 'character.location.z','character.isInBlueZone']]
df2=df[['_D', 'common.isGame', 'gameState.safetyZoneRadius','gameState.safetyZonePosition.x', 'gameState.safetyZonePosition.y', 'gameState.safetyZonePosition.z']]

C:\Users\PC\AppData\Local\Temp\ipykernel_18992\1579400500.py:2: DtypeWarning: Columns (24,26,30,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\data\tel0.csv')


In [3]:
pwd

'c:\\Users\\PC\\Desktop\\AI_Lab\\pubg'

In [4]:
cd tel_processing

c:\Users\PC\Desktop\AI_Lab\pubg\tel_processing


c:\Users\PC\Desktop\AI_Lab\pubg\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
#저장진행
df1.to_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\tel_processing\character.csv', index=False)
df2.to_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\tel_processing\gameState.csv', index=False)

#### 데이터 전처리

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97339 entries, 0 to 97338
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _D                      97339 non-null  object 
 1   character.name          49960 non-null  object 
 2   character.location.x    49960 non-null  float64
 3   character.location.y    49960 non-null  float64
 4   character.location.z    49960 non-null  float64
 5   character.isInBlueZone  49960 non-null  object 
dtypes: float64(3), object(3)
memory usage: 4.5+ MB


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97339 entries, 0 to 97338
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   _D                              97339 non-null  object 
 1   common.isGame                   97338 non-null  float64
 2   gameState.safetyZoneRadius      176 non-null    float64
 3   gameState.safetyZonePosition.x  176 non-null    float64
 4   gameState.safetyZonePosition.y  176 non-null    float64
 5   gameState.safetyZonePosition.z  176 non-null    float64
dtypes: float64(5), object(1)
memory usage: 4.5+ MB


In [8]:
df1 = df1.dropna()
df2 = df2.dropna()

In [9]:
# common.isGame = 자기장 페이즈 수치로 추정가능
df2[['_D', 'common.isGame', 'gameState.safetyZoneRadius']]['common.isGame'].unique()

array([0.1, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
       7. , 7.5, 8. , 8.5])

In [10]:
# 0.1 초기값 > 이후 int형에선 생존구역 정지 > float 에서 생존구역 중심좌표 이동 및 반지름 감소 시작
df2[30:40]

,_D,common.isGame,gameState.safetyZoneRadius,gameState.safetyZonePosition.x,gameState.safetyZonePosition.y,gameState.safetyZonePosition.z
27943,2023-12-01T11:25:16.690Z,1.0,581999.12500,408000.00000,408000.00000,0.0
28265,2023-12-01T11:25:26.696Z,1.0,581999.12500,408000.00000,408000.00000,0.0
28679,2023-12-01T11:25:36.680Z,1.0,581999.12500,408000.00000,408000.00000,0.0
29102,2023-12-01T11:25:46.721Z,1.5,568101.25000,409120.06250,402027.34375,0.0
29422,2023-12-01T11:25:56.705Z,1.5,554119.93750,410246.84375,396018.84375,0.0
29878,2023-12-01T11:26:06.693Z,1.5,540116.81250,411375.37500,390000.93750,0.0
30255,2023-12-01T11:26:16.664Z,1.5,526142.87500,412501.56250,383995.56250,0.0
30592,2023-12-01T11:26:26.708Z,1.5,512079.09375,413635.00000,377951.62500,0.0
30870,2023-12-01T11:26:36.674Z,1.5,498108.09375,414760.93750,371947.56250,0.0
31191,2023-12-01T11:26:46.677Z,1.5,484105.31250,415889.43750,365929.81250,0.0


In [11]:
#_D 시간데이터 정규표현식으로 전처리
df1['_D'] = pd.to_datetime(df1['_D']).dt.strftime('%H:%M:%S.%f').str[:-3]
df2['_D'] = pd.to_datetime(df2['_D']).dt.strftime('%H:%M:%S.%f').str[:-3]

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49960 entries, 5 to 97338
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _D                      49960 non-null  object 
 1   character.name          49960 non-null  object 
 2   character.location.x    49960 non-null  float64
 3   character.location.y    49960 non-null  float64
 4   character.location.z    49960 non-null  float64
 5   character.isInBlueZone  49960 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.7+ MB


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 14452 to 88333
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   _D                              176 non-null    object 
 1   common.isGame                   176 non-null    float64
 2   gameState.safetyZoneRadius      176 non-null    float64
 3   gameState.safetyZonePosition.x  176 non-null    float64
 4   gameState.safetyZonePosition.y  176 non-null    float64
 5   gameState.safetyZonePosition.z  176 non-null    float64
dtypes: float64(5), object(1)
memory usage: 9.6+ KB


In [14]:
df2[31:35]

,_D,common.isGame,gameState.safetyZoneRadius,gameState.safetyZonePosition.x,gameState.safetyZonePosition.y,gameState.safetyZonePosition.z
28265,11:25:26.696,1.0,581999.1250,408000.00000,408000.00000,0.0
28679,11:25:36.680,1.0,581999.1250,408000.00000,408000.00000,0.0
29102,11:25:46.721,1.5,568101.2500,409120.06250,402027.34375,0.0
29422,11:25:56.705,1.5,554119.9375,410246.84375,396018.84375,0.0


In [15]:
df2[58:62]

,_D,common.isGame,gameState.safetyZoneRadius,gameState.safetyZonePosition.x,gameState.safetyZonePosition.y,gameState.safetyZonePosition.z
37198,11:29:56.680,1.5,217880.84375,437345.00000,251518.906250,0.0
37423,11:30:06.724,1.5,203815.78125,438478.53125,245474.406250,0.0
37658,11:30:16.702,2.0,203699.68750,438487.87500,245424.515625,0.0
37987,11:30:26.749,2.0,203699.68750,438487.87500,245424.515625,0.0


- 1경기에 64명이 뛰는데 일치하는 값만 찾을수밖에 없다보니 24개 데이터만 나옴 + 초기값이 합쳐짐
- 전체 데이터로 확장시 scaatter plot은 그려볼 수 있을지 몰라도 의미 있는 데이터로 보긴 어려울듯함

다른 방법을 찾아야함

아이디어 : 안전구역 중심부를 구해서 
- 페이즈별로 자기장이 움직인 후 멈춤 = 페이즈 진행전 이미 안전구역이 설정됨

임시 용어정의
- bluezone : 자기장 외부로 
- whitecircle : 페이즈 진행시 줄어들기로 약속된 
- 생존구역 : bluezone 과 whitecircle 사이 

우선 gameState.safetyZoneRadius.1 값이 변하는 지점 찾아야 감이 올듯함

common.isGame 수치 변화에 따른 반지름 변화 시점 추측시작


In [16]:
df2_sample = df2[['_D','common.isGame','gameState.safetyZoneRadius']]

df2_start = df2_sample.drop_duplicates('common.isGame', keep='first')
df2_end = df2_sample.drop_duplicates('common.isGame', keep='last')

In [17]:
df2_start.head()

,_D,common.isGame,gameState.safetyZoneRadius
14452,11:20:16.798,0.1,581999.12500
16323,11:21:46.701,1.0,581999.12500
29102,11:25:46.721,1.5,568101.25000
37658,11:30:16.702,2.0,203699.68750
42306,11:31:46.669,2.5,196419.09375


In [18]:
df2_end.head()

,_D,common.isGame,gameState.safetyZoneRadius
15771,11:21:36.691,0.1,581999.125000
28679,11:25:36.680,1.0,581999.125000
37423,11:30:06.724,1.5,203815.781250
41969,11:31:36.666,2.0,203699.687500
46013,11:33:36.699,2.5,112364.085938


In [19]:
# df2_start, end 값 비교하여 볼수 있도록 df2_phase 구성
df2_phase = pd.merge(df2_start, df2_end, on='common.isGame')
df2_phase = df2_phase[['common.isGame','_D_x','_D_y']]

In [20]:
# 시간이 얼추 10초단위로 기록됨
df2_phase.columns = ['phase','start_time','end_time']
df2_phase

# 게임시작후 1분30초뒤 첫번째 자기장 결정 > 1.0 startime

,phase,start_time,end_time
0,0.1,11:20:16.798,11:21:36.691
1,1.0,11:21:46.701,11:25:36.680
2,1.5,11:25:46.721,11:30:06.724
3,2.0,11:30:16.702,11:31:36.666
4,2.5,11:31:46.669,11:33:36.699
5,3.0,11:33:46.718,11:34:36.730
6,3.5,11:34:46.743,11:36:36.663
7,4.0,11:36:46.726,11:37:36.674
8,4.5,11:37:46.719,11:39:36.730
9,5.0,11:39:46.697,11:40:36.658


In [21]:
print(df2_phase['phase'].unique())

[0.1 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5]


- 0.1은 게임 시작초반에 캐릭 생성 때 나타나는 데이터
- 나머지 [1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5] 는 페이즈로 볼수 있음

페이즈별 시간에 따라 판단 후 gameState 값 넣는 함수 만들기

In [22]:
# _D값 비교가능하게 float화하는 함수
def time_to_float(row):
    time_data = row['_D']
    hours, minutes, seconds = map(float, time_data.split(':'))
    time_data = hours * 3600 + minutes * 60 + seconds

    return  time_data

In [23]:
# 선수별 시간 데이터 : 수치상 비교가능하게 환산 후 list화
player_time = df1.swifter.apply(time_to_float, axis=1)
player_time_list = player_time.to_list()
player_time_list

Pandas Apply:  58%|█████▊    | 29120/49960 [00:00<00:00, 98305.47it/s]

Pandas Apply: 100%|██████████| 49960/49960 [00:00<00:00, 91310.45it/s]


[38721.247,
 38721.266,
 38721.283,
 38721.301,
 38721.323,
 38721.339,
 38721.367,
 38721.385,
 38721.403,
 38721.425,
 38721.444,
 38721.472,
 38721.491,
 38721.513,
 38721.535,
 38721.555,
 38721.571,
 38721.586,
 38721.603,
 38721.629,
 38721.643,
 38721.663,
 38721.685,
 38721.718,
 38721.734,
 38721.751,
 38721.766,
 38721.783,
 38721.802,
 38721.828,
 38721.86,
 38721.878,
 38721.894,
 38721.916,
 38721.939,
 38721.96,
 38721.99,
 38722.007,
 38722.04,
 38722.063,
 38722.103,
 38722.145,
 38723.004,
 38723.004,
 38723.004,
 38723.004,
 38723.761,
 38723.779,
 38723.795,
 38723.814,
 38723.832,
 38723.855,
 38723.873,
 38723.89,
 38723.908,
 38723.924,
 38723.94,
 38723.959,
 38723.975,
 38723.993,
 38724.011,
 38724.028,
 38724.045,
 38724.066,
 38724.084,
 38724.109,
 38724.349,
 38724.657,
 38724.658,
 38724.878,
 38724.878,
 38724.991,
 38724.991,
 38725.227,
 38725.32,
 38725.422,
 38725.519,
 38725.519,
 38725.9,
 38725.9,
 38726.336,
 38726.385,
 38726.647,
 38727.165,
 38

In [24]:
# 수치상 비교위해 환산
start_time = df2_start.swifter.apply(time_to_float, axis=1)
start_time_list = start_time.to_list()
start_time_list

Pandas Apply: 100%|██████████| 17/17 [00:00<00:00, 8521.95it/s]


[40816.798,
 40906.701,
 41146.721,
 41416.702,
 41506.669,
 41626.718,
 41686.743,
 41806.726,
 41866.719,
 41986.697,
 42046.732,
 42166.653,
 42226.674,
 42346.662,
 42406.653,
 42496.663,
 42556.66]

In [25]:
# 수치상 비교위해 환산
end_time = df2_end.swifter.apply(time_to_float, axis=1)
end_time_list = end_time.to_list()
end_time_list

Pandas Apply: 100%|██████████| 17/17 [00:00<00:00, 8522.97it/s]


[40896.691,
 41136.68,
 41406.724,
 41496.666,
 41616.699,
 41676.73,
 41796.663,
 41856.674,
 41976.73,
 42036.658,
 42156.654,
 42216.678,
 42336.661,
 42396.648,
 42486.659,
 42546.654,
 42566.644]

결국 처음 자기장이 정해지는 시간 기준으로 위 데이터를 날려야할지 고민
- why? 어차피 첫데이터가 수송기부터 있어서 사라짐
- 거리의 평균을 내버릴 때 수송기랑 자기장간 거리가 데이터 왜곡할듯함

방법 1. 안전지역 결정 이전 시점 제거
- 적게 시간 들이고 전처리가능
- but 낙하지점, 첫 탑승지점 데이터와의 중복 발생 > 방법 2 구현 못할시 사용

In [26]:
# 방법 1. prephase(안전지역 결정 이전전) 시점 제거함수 
def delete_prephase_1(row):
    player_time = row['_D']
    hours, minutes, seconds = map(float, player_time.split(':'))
    player_time = hours * 3600 + minutes * 60 + seconds

    return player_time>= start_time_list[1]
        
df1[df1.apply(delete_prephase_1,axis=1)]

,_D,character.name,character.location.x,character.location.y,character.location.z,character.isInBlueZone
16324,11:21:46.740,V7_Glaz,531114.81250,366788.906250,9432.629883,False
16325,11:21:46.740,V7_Glaz,531114.81250,366788.906250,9432.629883,False
16326,11:21:46.746,TL_ATongMuu,634363.50000,268241.218750,5799.199707,False
16327,11:21:46.767,NH_Nannnnn,515366.09375,479661.656250,14936.359375,False
16328,11:21:46.810,17_xwudd,371513.43750,423392.343750,6460.129883,False
...,...,...,...,...,...,...
97334,11:50:45.342,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False
97335,11:50:49.007,T5_RossitedJR,411438.90625,217228.031250,4704.060059,False
97336,11:50:52.825,T5_Thanad0l,413216.18750,213688.890625,4371.159668,False
97337,11:50:53.337,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False


방법 2. 플레이어 낙하시점 기준 이전 시점(이하 prephase) 제거
- 구현만 된다면 낙하 이후 운영을 면밀하게 살펴볼 수 있음
- 어렵지면 구현 시도

In [27]:
# 방법 2 진행전 낙하위치 및 시간 산출 위해 df3 구성

df = pd.read_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\data\tel0.csv')
df3=df[['_D','character.name', 'character.location.x', 'character.location.y', 'character.location.z','item.itemId']]

C:\Users\PC\AppData\Local\Temp\ipykernel_18992\489042967.py:3: DtypeWarning: Columns (24,26,30,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\data\tel0.csv')


In [28]:
# 기본적인 다듬기
df3 = df3.dropna()
df3['_D'] = pd.to_datetime(df3['_D']).dt.strftime('%H:%M:%S.%f').str[:-3]

In [29]:
# 전처리 결과 체크
df3.head()

,_D,character.name,character.location.x,character.location.y,character.location.z,item.itemId
14005,11:20:06.148,LG_Fludd,984378.0,381234.34375,150208.0,Item_Back_B_01_StartParachutePack_C
14006,11:20:06.148,LG_Fludd,984378.0,381234.34375,150208.0,Item_Back_B_01_StartParachutePack_C
14008,11:20:06.149,LG_Snakers,984378.0,381234.34375,150208.0,Item_Back_B_01_StartParachutePack_C
14009,11:20:06.149,LG_Snakers,984378.0,381234.34375,150208.0,Item_Back_B_01_StartParachutePack_C
14011,11:20:06.150,LG_Relo,984378.0,381234.34375,150208.0,Item_Back_B_01_StartParachutePack_C


In [30]:
# 낙하산만 사용한 값만 나오게 처리
df3 = df3[df3['item.itemId'] == 'Item_Back_B_01_StartParachutePack_C']

In [31]:
# item.itemId는 equip, unequip 총 2번씩 log가 남음
# 마지막 데이터만 남게하면 착지한 시점 및 시간 알 수 있음
df3 = df3.drop_duplicates(subset='character.name', keep='last')

In [32]:
# 64명의 선수별 낙하위치 및 시간 데이터 산출완료
df3

,_D,character.name,character.location.x,character.location.y,character.location.z,item.itemId
14819,11:21:03.132,TL_Jiaoyang,605780.87500,366935.437500,10457.629883,Item_Back_B_01_StartParachutePack_C
14849,11:21:06.992,TL_Lzqq,644191.00000,264852.031250,6414.707520,Item_Back_B_01_StartParachutePack_C
14881,11:21:09.653,V7_Gyumin,512450.31250,362163.375000,12157.847656,Item_Back_B_01_StartParachutePack_C
14883,11:21:09.685,V7_Tosi,513454.90625,361667.562500,12003.504883,Item_Back_B_01_StartParachutePack_C
14912,11:21:11.753,TL_Shen,564564.31250,298073.906250,12544.645508,Item_Back_B_01_StartParachutePack_C
...,...,...,...,...,...,...
18004,11:22:12.659,DNW_Inonix,293784.43750,486528.656250,8965.875977,Item_Back_B_01_StartParachutePack_C
51841,11:35:17.545,SSG_PaG3,402796.37500,166575.718750,4366.081055,Item_Back_B_01_StartParachutePack_C
52318,11:35:21.281,SSG_Roth,418312.25000,189524.671875,6115.812012,Item_Back_B_01_StartParachutePack_C
52425,11:35:23.475,SSG_Sharpshot4K,416697.43750,186477.484375,6085.400391,Item_Back_B_01_StartParachutePack_C


In [33]:
# 낙하시점 : 수치상 비교가능하게 환산 후 list화
landing_time = df3.swifter.apply(time_to_float, axis=1)
landing_time_list = landing_time.to_list()
landing_time_list

Pandas Apply: 100%|██████████| 64/64 [00:00<00:00, 21402.92it/s]


[40863.132,
 40866.992,
 40869.653,
 40869.685,
 40871.753,
 40871.932,
 40872.05,
 40873.468,
 40873.911,
 40874.453,
 40874.716,
 40877.759,
 40877.945,
 40878.049,
 40878.142,
 40878.675,
 40879.346,
 40880.098,
 40881.139,
 40881.637,
 40882.308,
 40882.979,
 40883.485,
 40885.51,
 40885.911,
 40886.773,
 40887.714,
 40887.951,
 40889.951,
 40890.418,
 40890.423,
 40892.077,
 40892.189,
 40893.784,
 40893.897,
 40894.222,
 40894.451,
 40895.248,
 40895.539,
 40896.662,
 40896.936,
 40896.937,
 40897.194,
 40898.304,
 40898.359,
 40899.018,
 40900.788,
 40900.856,
 40901.237,
 40902.201,
 40902.959,
 40903.851,
 40906.078,
 40907.32,
 40915.349,
 40919.848,
 40921.663,
 40925.823,
 40926.952,
 40932.659,
 41717.545,
 41721.281,
 41723.475,
 41725.385]

In [34]:
# 개별 선수의 이름으로 착지시간 산출 가능한지 테스트
player_name='SSG_PiXeL1K'
landing_time = df3[df3['character.name'].str.contains(player_name)]['_D'].to_list()[0]
landing_time

'11:35:25.385'

In [35]:
# prephase 제거하는 함수

def delete_prephase(row):
    player_time = row['_D']
    player_name = row['character.name']
    hours, minutes, seconds = map(float, player_time.split(':'))
    player_time = hours * 3600 + minutes * 60 + seconds
    
    landing_time = df3[df3['character.name'].str.contains(player_name)]['_D'].to_list()[0]
    hours, minutes, seconds = map(float, landing_time.split(':'))
    landing_time = hours * 3600 + minutes * 60 + seconds

    for name in df3['character.name']:
        if name == row['character.name']:
            return player_time >= landing_time

In [36]:
df1

,_D,character.name,character.location.x,character.location.y,character.location.z,character.isInBlueZone
5,10:45:21.247,NH_MMing,125273.140625,703364.812500,10763.201172,False
7,10:45:21.266,GEN_Seongjang,124567.429688,701831.875000,10763.201172,False
9,10:45:21.283,LG_Snakers,125034.640625,704196.187500,10763.201172,False
11,10:45:21.301,CES_HaiSaki,125502.757812,702065.500000,10763.201172,False
14,10:45:21.323,Tian_Zyy,124901.148438,702914.312500,10763.201172,False
...,...,...,...,...,...,...
97334,11:50:45.342,T5_Hatsawat,404608.062500,215308.390625,6225.689941,False
97335,11:50:49.007,T5_RossitedJR,411438.906250,217228.031250,4704.060059,False
97336,11:50:52.825,T5_Thanad0l,413216.187500,213688.890625,4371.159668,False
97337,11:50:53.337,T5_Hatsawat,404608.062500,215308.390625,6225.689941,False


In [37]:
df1 = df1[df1.swifter.apply(delete_prephase,axis=1)]

Pandas Apply: 100%|██████████| 49960/49960 [00:25<00:00, 1980.28it/s]


In [38]:
df1

,_D,character.name,character.location.x,character.location.y,character.location.z,character.isInBlueZone
14819,11:21:03.132,TL_Jiaoyang,605780.87500,366935.437500,10457.629883,False
14820,11:21:03.133,TL_Jiaoyang,605888.00000,366900.062500,10551.200195,False
14822,11:21:03.651,TL_Jiaoyang,605379.68750,367114.625000,9876.229492,False
14835,11:21:05.326,TL_Jiaoyang,605124.12500,367265.593750,9903.726562,False
14837,11:21:05.448,TL_Jiaoyang,605124.12500,367265.593750,9903.726562,False
...,...,...,...,...,...,...
97334,11:50:45.342,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False
97335,11:50:49.007,T5_RossitedJR,411438.90625,217228.031250,4704.060059,False
97336,11:50:52.825,T5_Thanad0l,413216.18750,213688.890625,4371.159668,False
97337,11:50:53.337,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False


방법 2로 산출가능하여 하기로 결정. 
- 연산시간이 좀 걸리지만 실효성 있는 데이터가 나을듯

In [39]:
# 0.1 과 1.0 페이즈 - 1분 30초 차이 = 최초 자기장 설정 전 시간과 동일 = phase로 볼 수 있는 근거  
df2_phase

,phase,start_time,end_time
0,0.1,11:20:16.798,11:21:36.691
1,1.0,11:21:46.701,11:25:36.680
2,1.5,11:25:46.721,11:30:06.724
3,2.0,11:30:16.702,11:31:36.666
4,2.5,11:31:46.669,11:33:36.699
5,3.0,11:33:46.718,11:34:36.730
6,3.5,11:34:46.743,11:36:36.663
7,4.0,11:36:46.726,11:37:36.674
8,4.5,11:37:46.719,11:39:36.730
9,5.0,11:39:46.697,11:40:36.658


In [40]:
# 수치상 비교가능하게 환산 후 list화 해둔 것들 재활용
print(player_time_list)
print(start_time_list)
print(end_time_list)

[38721.247, 38721.266, 38721.283, 38721.301, 38721.323, 38721.339, 38721.367, 38721.385, 38721.403, 38721.425, 38721.444, 38721.472, 38721.491, 38721.513, 38721.535, 38721.555, 38721.571, 38721.586, 38721.603, 38721.629, 38721.643, 38721.663, 38721.685, 38721.718, 38721.734, 38721.751, 38721.766, 38721.783, 38721.802, 38721.828, 38721.86, 38721.878, 38721.894, 38721.916, 38721.939, 38721.96, 38721.99, 38722.007, 38722.04, 38722.063, 38722.103, 38722.145, 38723.004, 38723.004, 38723.004, 38723.004, 38723.761, 38723.779, 38723.795, 38723.814, 38723.832, 38723.855, 38723.873, 38723.89, 38723.908, 38723.924, 38723.94, 38723.959, 38723.975, 38723.993, 38724.011, 38724.028, 38724.045, 38724.066, 38724.084, 38724.109, 38724.349, 38724.657, 38724.658, 38724.878, 38724.878, 38724.991, 38724.991, 38725.227, 38725.32, 38725.422, 38725.519, 38725.519, 38725.9, 38725.9, 38726.336, 38726.385, 38726.647, 38727.165, 38727.233, 38727.359, 38727.495, 38727.742, 38727.877, 38728.148, 38728.467, 38728.662

In [41]:
 # 선수별 시간값과 페이즈별 시작시간 값을 비교하여 phase, 하얀 원 중심좌표, 반지름 데이터를 df1에 추가하는 함수
def add_gameStates(row):

    player_time = row['_D']
    hours, minutes, seconds = map(float, player_time.split(':'))
    player_time = hours * 3600 + minutes * 60 + seconds

    safez_phases = [df2_phase['start_time'][i] for i in range(3, 16, 2)]
    safez_phase_times = [hours * 3600 + minutes * 60 + seconds for phase in safez_phases for hours, minutes, seconds in [map(float, phase.split(':'))]]

   
    # 예) 2페이즈 start_time 시간 이전 선수 위치는 아직 1페이즈 진행중이라 할 수 있음
    #     이는 phase = 1.0, 1페이즈의 하얀 원 중심좌표 및 반지름 = 2페이즈의 페이즈 준비시간시 정지해있는 원의 좌표 및 반지름 의미
    # 위 내용을 바탕으로 1페이즈, 2페이즈, ... 7페이즈 계산함
    for i in range(0,len(safez_phase_times)):
        if player_time <= safez_phase_times[i]:
            phase = float(df2[df2['_D'].str.contains(safez_phases[i])]['common.isGame'].to_list()[0])-1.0
            whitecircle_location_x = df2[df2['_D'].str.contains(safez_phases[i])]['gameState.safetyZonePosition.x'].to_list()[0]
            whitecircle_location_y = df2[df2['_D'].str.contains(safez_phases[i])]['gameState.safetyZonePosition.y'].to_list()[0]
            whitecircle_radius = df2[df2['_D'].str.contains(safez_phases[i])]['gameState.safetyZoneRadius'].to_list()[0]

            return phase, whitecircle_location_x, whitecircle_location_y, whitecircle_radius

    # 이후 기준 하얀원 값이 없지만, 어차피 원이 작아서 마지막 phase data로 대신 설정    
    if player_time > safez_phase_times[6]:
        phase = float(df2[df2['_D'].str.contains(safez_phases[6])]['common.isGame'].to_list()[0])-1.0
        whitecircle_location_x = df2[df2['_D'].str.contains(safez_phases[6])]['gameState.safetyZonePosition.x'].to_list()[0]
        whitecircle_location_y = df2[df2['_D'].str.contains(safez_phases[6])]['gameState.safetyZonePosition.y'].to_list()[0]
        whitecircle_radius = df2[df2['_D'].str.contains(safez_phases[6])]['gameState.safetyZoneRadius'].to_list()[0]

        return phase, whitecircle_location_x, whitecircle_location_y, whitecircle_radius

In [42]:
df1[['phase', 'whitecircle_location_x', 'whitecircle_location_y', 'whitecircle_radius']] = df1.swifter.apply(add_gameStates, axis=1, result_type='expand')

Pandas Apply: 100%|██████████| 33833/33833 [00:55<00:00, 608.64it/s]
C:\Users\PC\AppData\Local\Temp\ipykernel_18992\3239937555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['phase', 'whitecircle_location_x', 'whitecircle_location_y', 'whitecircle_radius']] = df1.swifter.apply(add_gameStates, axis=1, result_type='expand')
C:\Users\PC\AppData\Local\Temp\ipykernel_18992\3239937555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['phase', 'whitecircle_location_x', 'whitecircle_location_y', 'w

In [43]:
df1

,_D,character.name,character.location.x,character.location.y,character.location.z,character.isInBlueZone,phase,whitecircle_location_x,whitecircle_location_y,whitecircle_radius
14819,11:21:03.132,TL_Jiaoyang,605780.87500,366935.437500,10457.629883,False,1.0,438487.87500,245424.515625,203699.687500
14820,11:21:03.133,TL_Jiaoyang,605888.00000,366900.062500,10551.200195,False,1.0,438487.87500,245424.515625,203699.687500
14822,11:21:03.651,TL_Jiaoyang,605379.68750,367114.625000,9876.229492,False,1.0,438487.87500,245424.515625,203699.687500
14835,11:21:05.326,TL_Jiaoyang,605124.12500,367265.593750,9903.726562,False,1.0,438487.87500,245424.515625,203699.687500
14837,11:21:05.448,TL_Jiaoyang,605124.12500,367265.593750,9903.726562,False,1.0,438487.87500,245424.515625,203699.687500
...,...,...,...,...,...,...,...,...,...,...
97334,11:50:45.342,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False,7.0,410221.46875,213685.875000,11076.323242
97335,11:50:49.007,T5_RossitedJR,411438.90625,217228.031250,4704.060059,False,7.0,410221.46875,213685.875000,11076.323242
97336,11:50:52.825,T5_Thanad0l,413216.18750,213688.890625,4371.159668,False,7.0,410221.46875,213685.875000,11076.323242
97337,11:50:53.337,T5_Hatsawat,404608.06250,215308.390625,6225.689941,False,7.0,410221.46875,213685.875000,11076.323242


In [44]:
df1['whitecircle_location_x'].unique()

array([438487.875  , 373675.71875, 393699.125  , 405977.71875,
       410590.34375, 412775.46875, 410221.46875])

In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33833 entries, 14819 to 97338
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _D                      33833 non-null  object 
 1   character.name          33833 non-null  object 
 2   character.location.x    33833 non-null  float64
 3   character.location.y    33833 non-null  float64
 4   character.location.z    33833 non-null  float64
 5   character.isInBlueZone  33833 non-null  object 
 6   phase                   33833 non-null  float64
 7   whitecircle_location_x  33833 non-null  float64
 8   whitecircle_location_y  33833 non-null  float64
 9   whitecircle_radius      33833 non-null  float64
dtypes: float64(7), object(3)
memory usage: 3.8+ MB


- 매치당 1분30초~2분정도 소요 x 18 매치 = 36분 정도 소요

- 맵별로 분석하면 더 적을듯 : 에란겔 6, 미라마6, 비켄디 3, 태이고 3 (각각 소요시각 12분, 12분, 6분, 6분) 

우선 첫 낙하위치 및 탑승위치 기준으로 1번째 자기장 좌표 유클리드 거리 계산해보면서 규칙 찾고, 여기에 적용할 예정

In [46]:
# 데이터 저장 

df1.to_csv(r'C:\Users\PC\Desktop\AI_Lab\pubg\preprocessed\character_location_per_phase.csv', index=False)